<a href="https://colab.research.google.com/github/tsangrebecca/BloomTech/blob/main/Sprint7/Module2/ExploreTabularDataForSupervisedMachineLearning_JoinRelationalDataForSupervisedMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Load in earthquake data sets
import pandas as pd

cols_set1 = ['Evid', 'Magnitude', 'Magnitude Type', 'Time UTC', 'Lat', 'Lon', 'Depth Km']
eq_set1 = pd.read_csv('pnsn_eqlist.csv', usecols=cols_set1)
# have to grab files from local drive in Sprint 7 folder

cols_set2 = ['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'net', 'id']
eq_set2 = pd.read_csv('usgs_eqlist.csv', usecols=cols_set2)

display(eq_set1.head())
display(eq_set2.head())

,Evid,Magnitude,Magnitude Type,Time UTC,Lat,Lon,Depth Km
0,61902967,1.6,l,2023/03/17 22:22:06,41.9735,-119.9800,-0.7
1,61902957,0.9,l,2023/03/17 21:57:56,48.4250,-123.5672,-0.3
2,61902942,0.9,l,2023/03/17 18:03:07,47.5310,-123.9540,29.0
3,61902932,0.8,l,2023/03/17 17:23:15,46.4353,-122.3502,18.3
4,61902892,1.0,l,2023/03/17 11:34:08,49.0520,-119.3693,0.6


,time,latitude,longitude,depth,mag,magType,net,id
0,2023-09-28T15:49:58.761Z,59.947000,-153.346200,133.00,2.10,ml,ak,ak023cgfv6iq
1,2023-09-28T15:37:58.350Z,32.730833,-115.552667,11.12,1.96,ml,ci,ci40573880
2,2023-09-28T15:35:11.810Z,35.882833,-117.688500,2.46,1.08,ml,ci,ci40573872
3,2023-09-28T15:32:25.797Z,62.166300,-148.508600,37.20,0.90,ml,ak,ak023cgfrgsz
4,2023-09-28T15:26:22.610Z,34.520333,-118.098500,9.28,1.67,ml,ci,ci40573864


We can see that each list of earthquakes has an identification column (Evid and id). If we looked at more rows, we would see that there is some overlap. In order to combine these two tables into one, we need to do a bit of cleaning so that we can compare the identification columns.

In [10]:
# Clean up eq_set2 id column
# (strip the 'uw' or 'us' from the number)
eq_set2['id'] = eq_set2['id'].map(lambda x: x.lstrip('uws'))
# only some of the 2020 data are overlapping

In [11]:
# Add column with the network code

eq_set1['net'] = 'uw'

# Rename columns
new_cols = ['id', 'mag','magType','time','latitude','longitude','depth','net']
eq_set1.columns = new_cols

eq_set1.head()

,id,mag,magType,time,latitude,longitude,depth,net
0,61902967,1.6,l,2023/03/17 22:22:06,41.9735,-119.9800,-0.7,uw
1,61902957,0.9,l,2023/03/17 21:57:56,48.4250,-123.5672,-0.3,uw
2,61902942,0.9,l,2023/03/17 18:03:07,47.5310,-123.9540,29.0,uw
3,61902932,0.8,l,2023/03/17 17:23:15,46.4353,-122.3502,18.3,uw
4,61902892,1.0,l,2023/03/17 11:34:08,49.0520,-119.3693,0.6,uw


We also need to check if the id column is of the same type for both DataFrames and change it if it's not.

In [12]:
# Check data types using .dtype
print('Data type for eq_set1: ', eq_set1['id'].dtype)
print('Data type for eq_set2: ', eq_set2['id'].dtype)

# Change data type using astype('type_i_want')
eq_set1['id'] = eq_set1['id'].astype('str')
print('Data type for eq_set1: ', eq_set1['id'].dtype)

Data type for eq_set1:  int64
Data type for eq_set2:  object
Data type for eq_set1:  object


In [13]:
# Take a look at our work
display(eq_set1.head())
display(eq_set2.head())

,id,mag,magType,time,latitude,longitude,depth,net
0,61902967,1.6,l,2023/03/17 22:22:06,41.9735,-119.9800,-0.7,uw
1,61902957,0.9,l,2023/03/17 21:57:56,48.4250,-123.5672,-0.3,uw
2,61902942,0.9,l,2023/03/17 18:03:07,47.5310,-123.9540,29.0,uw
3,61902932,0.8,l,2023/03/17 17:23:15,46.4353,-122.3502,18.3,uw
4,61902892,1.0,l,2023/03/17 11:34:08,49.0520,-119.3693,0.6,uw


,time,latitude,longitude,depth,mag,magType,net,id
0,2023-09-28T15:49:58.761Z,59.947000,-153.346200,133.00,2.10,ml,ak,ak023cgfv6iq
1,2023-09-28T15:37:58.350Z,32.730833,-115.552667,11.12,1.96,ml,ci,ci40573880
2,2023-09-28T15:35:11.810Z,35.882833,-117.688500,2.46,1.08,ml,ci,ci40573872
3,2023-09-28T15:32:25.797Z,62.166300,-148.508600,37.20,0.90,ml,ak,ak023cgfrgsz
4,2023-09-28T15:26:22.610Z,34.520333,-118.098500,9.28,1.67,ml,ci,ci40573864


Now we can combine the two sets. First, we need to think about how to join the DataFrames. In general, we should keep all of the data if possible. In this case, the columns in each DataFrame are the same, so we're essentially concatenating the two DataFrames. After we do that, we'll check for duplication in the event id and remove any duplicates.

In [14]:
# Concatenate eq_set1 and eq_set2
eq_set_all = pd.concat([eq_set1, eq_set2])
print('Size of DataFrame before dropping duplicates: ', eq_set_all.shape)

# Remove duplicate event id numbers
eq_set_all.drop_duplicates(subset='id', inplace=True)
print('Size of DataFrame after dropping duplicates: ', eq_set_all.shape)

Size of DataFrame before dropping duplicates:  (19540, 8)
Size of DataFrame after dropping duplicates:  (19540, 8)
